In [1]:
import pandas as pd
df = pd.read_csv('titanic.csv')

In [2]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
x = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

In [6]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [7]:
x.head()

,Pclass,SibSp,Parch,Fare
0,3,1,0,7.2500
1,1,1,0,71.2833
2,3,0,0,7.9250
3,1,1,0,53.1000
4,3,0,0,8.0500


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25)

In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7040358744394619

### Cross validation - k fold method

In [10]:
from sklearn.model_selection import cross_val_score, KFold
model = LogisticRegression()
kf = KFold(n_splits = 5)
score = cross_val_score(model,x,y,cv=kf)
print('Cross validation scores are {}'.format(score))

Cross validation scores are [0.62569832 0.66853933 0.66853933 0.69662921 0.75280899]


In [11]:
print ("Average cross val score : {}".format(score.mean()))

Average cross val score : 0.6824430355909861


### Tuning

In [12]:
algorithms = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

In [13]:
for algo in algorithms:
    score = cross_val_score(LogisticRegression(max_iter= 4000, solver= algo), x, y, cv= kf, scoring="accuracy")
    print(f'Average score({algo}): {"{:.3f}".format(score.mean())}')

Average score(newton-cg): 0.682
Average score(lbfgs): 0.682
Average score(liblinear): 0.682
Average score(sag): 0.669
Average score(saga): 0.669


In [14]:
from sklearn import tree
max_depth = [1,2,3,4,5,6,7,8,9,10]

for val in max_depth:
    score = cross_val_score(tree.DecisionTreeClassifier(max_depth= val), x, y, cv= kf, scoring="accuracy")
    print(f'Average score({val}): {"{:.3f}".format(score.mean())}')

Average score(1): 0.668
Average score(2): 0.694
Average score(3): 0.705
Average score(4): 0.707
Average score(5): 0.712
Average score(6): 0.722
Average score(7): 0.714
Average score(8): 0.712
Average score(9): 0.706
Average score(10): 0.699


### Grid search

In [15]:
parameters ={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
     'criterion' : ['gini', 'entropy'],
     'n_estimators': [50,100,200,400,600]}

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
from sklearn import ensemble
grid_search = GridSearchCV(
    ensemble.RandomForestClassifier(), 
    parameters, 
    cv=5,
    scoring='accuracy',n_jobs=-1)

In [18]:
grid_result = grid_search.fit(x_train, y_train)

In [19]:
grid_result

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                         'n_estimators': [50, 100, 200, 400, 600]},
             scoring='accuracy')

In [20]:
print('best params:',grid_result.best_params_)
print('best score:', grid_result.best_score_)

best params: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 50}
best score: 0.6930647514308159


## random search cv and hyperopt